In [1]:
!pip install openai
!pip install langchain
!pip install tqdm
!pip install chromadb
!pip install tiktoken
!pip install sentence_transformers

In [2]:
!pip install langchain_community

In [15]:
from langchain.chat_models import ChatOpenAI
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('')

In [16]:
# rag 위한 파일 준비
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt",
    filename="state_of_the_union.txt"
)

('state_of_the_union.txt', <http.client.HTTPMessage at 0x7d248402ffd0>)

In [17]:
from tqdm import tqdm



Simple TextLoader 구현해보기

In [18]:
class SimpleTextLoader:
    def __init__(self, file_path):
        self.file_path = file_path


    def load(self):
        text = '' #파일에서 읽은 데이터 저장 변수
        with open(self.file_path, 'r', encoding='utf-8') as f :
            text = f.read()
        return text


SimpleCharacterTextSplitter 구현

In [19]:
# SimpleCharacterTextSplitter 구현

class SimpleCharacterTextSplitter:
    def __init__(self, chunk_size, chunk_overlap, separator_pattern = '\n\n'):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separator_pattern = separator_pattern

    #문장 분할 함수(메소드: 클래스내 함수)
    def split_document(self, documents):

        #파일 전체내용(documents) >> 문단 단위로 나누기
        splits = documents.split(self.separator_pattern) #분할 기준 : separator_pattern

        chunks = [] #최종적으로 생성될 chunks 저장
        current_chunk = splits[0]
        #분할된 첫번째 문단 >> 초기 chunk로 저장

        for split in tqdm(splits[1:], desc='spliting...'):
            # splits[1:] >> current_chunk 다음 문단
            # 구분자가 chunk_size를 초과했는지 확인
            if len(current_chunk) + len(split) + len(self.separator_pattern) > self.chunk_size:
                chunks.append(current_chunk.strip())
                current_chunk = split
                #새로운 chunk의 시작
            else:
                 #초과하지 않으면 >> 헌재 청크(current_chunk)에 구분자 + 다음 문단 추가
                 current_chunk += self.separator_pattern
                 current_chunk += split

        #마지막 청크 추가
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks



SimpleOpenAIEmbeddings 구현해보기

In [20]:
from openai import OpenAI

class SimpleOpenAIEmbeddings:
    def embed_query(self, text):
        client = OpenAI()
        response = client.embeddings.create(
            input = text,
            model = 'text-embedding-ada-002'
        )
        return response.data[0].embedding


SimpleRetriever 구현

In [21]:
class SimpleRetriever:

    def __init__(self, vector_store, k=4):
        self.vector_store = vector_store
        self.k = k

    def get_relevant_documents(self, query):
        return self.vector_store.similarity_search(query, self.k)


SimpleVectorStore(구현)

In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class SimpleVectorStore:
    def __init__(self, docs, embedding):
        self.embedding = embedding
        self.documents = [] #문서 내용을 저장할 리스트
        self.vectors = [] # 문서의 벡터 저장할 리스트

        for doc in tqdm(docs, desc = 'embedding...'):
            self.documents.append(doc)
            vector = self.embedding.embed_query(doc) #문서 >> 벡터(숫자) 변환
            self.vectors.append(vector)

    #유사도 검색 함수(메소드)
    def similarity_search(self, query, k=4) : #가장 유사한 4개 문서 변환
        query_vector = self.embedding.embed_query(query) #query >> 벡터(숫자) 변환

        if not self.vectors: #저장된 벡터가 없으면 >> 빈 리스트 반환
            return []

        similarities = cosine_similarity([query_vector], self.vectors)[0]
        #쿼리 벡터(query vector) : 저장된 벡터 간 코사인 유사도 계산
        sorted_doc_similarities = sorted(zip(self.documents, similarities), key = lambda x : x[1], reverse=True)
        # zip(문서, 유사도) tuple 형태 >> key x[1](유사도 기준) 내림차순
        return sorted_doc_similarities[:k] #유사도가 큰 상위 4개만 반환

    def as_retriever(self, k = 4):
        return SimpleRetriever(self, k)




In [23]:
path = '/content/state_of_the_union.txt'

raw_documents = SimpleTextLoader(path).load()
text_splitter = SimpleCharacterTextSplitter(chunk_size = 1000, chunk_overlap= 0 )
documents = text_splitter.split_document(raw_documents)

spliting...: 100%|██████████| 358/358 [00:00<00:00, 658290.59it/s]


In [24]:
documents[0]

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.'

In [25]:
len(documents)

42

In [26]:
db = SimpleVectorStore(documents, SimpleOpenAIEmbeddings())

embedding...: 100%|██████████| 42/42 [00:09<00:00,  4.20it/s]


In [27]:
query = "What did the president say about Ketanji Brown Jackson"

In [28]:
docs = db.similarity_search(query)
docs

[('Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  0.8152000921997997),
 ('A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’

In [29]:
print(docs[0])

('Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', 0.8152000921997997)


In [30]:
docs[0][0] # 문장

'Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'

In [31]:
docs[0][1] # 유사도


0.8152000921997997

한글 VectorStore

In [32]:
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/puzzlet/constitution-kr/master/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%ED%97%8C%EB%B2%95.txt",
    filename="korea_constitution.txt"
)

('korea_constitution.txt', <http.client.HTTPMessage at 0x7d2483797880>)

In [33]:
#1. 데이터 가져오기(load)
raw_documents = SimpleTextLoader('/content/korea_constitution.txt').load()

# 2. 텍스트 분할(text split)
text_splitter = SimpleCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_document(raw_documents)

spliting...: 100%|██████████| 152/152 [00:00<00:00, 582223.02it/s]


In [34]:
documents

['전문\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.\n\n제1장 총강\n\n제1조 ① 대한민국은 민주공화국이다.\n② 대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.\n\n제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.\n② 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.\n\n제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.\n\n제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.\n\n제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.\n② 국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.\n\n제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.\n② 외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.\n\n제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.\n② 공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.',
 '제8조 ① 정당의 설

In [35]:
print(documents)

['전문\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.\n\n제1장 총강\n\n제1조 ① 대한민국은 민주공화국이다.\n② 대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.\n\n제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.\n② 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.\n\n제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.\n\n제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.\n\n제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.\n② 국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.\n\n제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.\n② 외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.\n\n제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.\n② 공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.', '제8조 ① 정당의 설립

In [36]:
# Vector Store 구축 (embedding 해서 넣기)
db = SimpleVectorStore(documents, SimpleOpenAIEmbeddings())

embedding...: 100%|██████████| 22/22 [00:05<00:00,  3.95it/s]


In [37]:
# 쿼리 (질의) / 유사도 기반

query = "대통령 임기는?"
docs = db.similarity_search(query)

In [38]:
docs

[('제65조 ① 대통령·국무총리·국무위원·행정각부의 장·헌법재판소 재판관·법관·중앙선거관리위원회 위원·감사원장·감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의 소추를 의결할 수 있다.\n② 제1항의 탄핵소추는 국회재적의원 3분의 1 이상의 발의가 있어야 하며, 그 의결은 국회재적의원 과반수의 찬성이 있어야 한다. 다만, 대통령에 대한 탄핵소추는 국회재적의원 과반수의 발의와 국회재적의원 3분의 2 이상의 찬성이 있어야 한다.\n③ 탄핵소추의 의결을 받은 자는 탄핵심판이 있을 때까지 그 권한행사가 정지된다.\n④ 탄핵결정은 공직으로부터 파면함에 그친다. 그러나, 이에 의하여 민사상이나 형사상의 책임이 면제되지는 아니한다.\n\n제4장 정부\n제1절 대통령\n\n제66조 ① 대통령은 국가의 원수이며, 외국에 대하여 국가를 대표한다.\n② 대통령은 국가의 독립·영토의 보전·국가의 계속성과 헌법을 수호할 책무를 진다.\n③ 대통령은 조국의 평화적 통일을 위한 성실한 의무를 진다.\n④ 행정권은 대통령을 수반으로 하는 정부에 속한다.\n\n제67조 ① 대통령은 국민의 보통·평등·직접·비밀선거에 의하여 선출한다.\n② 제1항의 선거에 있어서 최고득표자가 2인 이상인 때에는 국회의 재적의원 과반수가 출석한 공개회의에서 다수표를 얻은 자를 당선자로 한다.\n③ 대통령후보자가 1인일 때에는 그 득표수가 선거권자 총수의 3분의 1 이상이 아니면 대통령으로 당선될 수 없다.\n④ 대통령으로 선거될 수 있는 자는 국회의원의 피선거권이 있고 선거일 현재 40세에 달하여야 한다.\n⑤ 대통령의 선거에 관한 사항은 법률로 정한다.\n\n제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8401072678447714),
 ('제81조 대통령은 국

벡터스토어 튜닝하기

In [39]:
#1. 데이터 가져오기(load)
raw_documents = SimpleTextLoader('/content/korea_constitution.txt').load()

# 2. 텍스트 분할(text split)
text_splitter = SimpleCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
documents = text_splitter.split_document(raw_documents)

spliting...: 100%|██████████| 152/152 [00:00<00:00, 638172.38it/s]


In [40]:
len(documents)

137

In [41]:
# Vector Store 구축 (embedding 해서 넣기)
db = SimpleVectorStore(documents, SimpleOpenAIEmbeddings())

embedding...: 100%|██████████| 137/137 [00:32<00:00,  4.21it/s]


In [42]:
# 쿼리 (질의)

query = "대통령 임기는 몇년인가?"
docs = db.similarity_search(query)

In [43]:
docs

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8763224902792133),
 ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8637671939561062),
 ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.',
  0.8513985590275148),
 ('제42조 국회의원의 임기는 4년으로 한다.\n\n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.',
  0.8471420496591688)]

검색기 만들기

In [44]:
retriever = db.as_retriever()

In [45]:
unique_docs = retriever.get_relevant_documents(query)
unique_docs

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8763224902792133),
 ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8637671939561062),
 ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.',
  0.8513985590275148),
 ('제42조 국회의원의 임기는 4년으로 한다.\n\n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.',
  0.8471420496591688)]

챗봇 만들기

In [46]:
import openai

system_prompt_template = ("You are a helpful assistant. "
                          "Based on the following content, "
                          "kindly and comprehensively respond to user questions. write in Korean."
                          "[Content]"
                          "{content}"
                          "")

class SimpleRetrievalQA():
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, query):
        docs = self.retriever.get_relevant_documents(query)
        print(docs)

        for i, doc in enumerate(docs):
            print("[#"+str(i) + "]", doc[1])
            print(doc[0])

        completion = openai.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = [
                {"role" : "system", "content": system_prompt_template},
                {"role" : "user", "content": query}
            ]
            ) #생성(G)

        return completion.choices[0].message.content


In [47]:
chain = SimpleRetrievalQA(retriever)
answer = chain.invoke("대통령 임기는?")

print()
print(">>",answer) # 실제 챗봇답변

[('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.', 0.8562506822849905), ('제4장 정부\n제1절 대통령', 0.8523114897668129), ('제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.\n"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."', 0.8514400510469455), ('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8502293529664914)]
[#0] 0.8562506822849905
제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.
② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.
[#1] 0.8523114897668129
제4장 정부
제1절 대통령
[#2] 0.8514400510469455
제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.
"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."
[#3] 0.8502293529664914
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.

>> 대통령의 임기는 한국 헌법에 따라서 5년입니다. 대한민국 헌법 제70조에 따르면 대통령은 선거에 의하여 선출되며, 임기는 5년으로 하되, 연임할 수 없습니다. 이러한 규정에 따라 대통령은 5년 동안 직무를 수행하며, 한 차례만 재선거를 통해 연임될 수 있습니다.


In [48]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

while True:
    user_message = input("user >>")
    if user_message.lower() == 'quit':
        break
    ai_message = chat_with_user(user_message)
    print(f'AI > , {ai_message}')

user >>대통령의 임기는?
[('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.', 0.8629479744559038), ('제4장 정부\n제1절 대통령', 0.8561093196233112), ('제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.\n"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."', 0.8561079259617459), ('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8538923553920164)]
[#0] 0.8629479744559038
제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.
② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.
[#1] 0.8561093196233112
제4장 정부
제1절 대통령
[#2] 0.8561079259617459
제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.
"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."
[#3] 0.8538923553920164
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
AI > , 대통령의 임기는 5년입니다. 처음으로 대통령이 선출되어 취임한 날을 기준으로 5년 후까지가 임기이며, 연임은 최대 한 번까지 가능합니다.
user >>quit


In [49]:
#ai 답변만 출력하기

retriever = db.as_retriever(k=3)
chain = SimpleRetrievalQA(retriever)
answer = chain.invoke('국회의원의 임기는?')

print(">>", answer)

[('제42조 국회의원의 임기는 4년으로 한다.\n\n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.', 0.8675928142694377), ('제63조 ① 국회는 국무총리 또는 국무위원의 해임을 대통령에게 건의할 수 있다.\n② 제1항의 해임건의는 국회재적의원 3분의 1 이상의 발의에 의하여 국회재적의원 과반수의 찬성이 있어야 한다.', 0.8654174907865853), ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.', 0.860796819770927)]
[#0] 0.8675928142694377
제42조 국회의원의 임기는 4년으로 한다.

제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
[#1] 0.8654174907865853
제63조 ① 국회는 국무총리 또는 국무위원의 해임을 대통령에게 건의할 수 있다.
② 제1항의 해임건의는 국회재적의원 3분의 1 이상의 발의에 의하여 국회재적의원 과반수의 찬성이 있어야 한다.
[#2] 0.860796819770927
제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.
② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.
>> 국회의원의 임기는 4년입니다. 국회의원은 선거를 통해 선출되며 임기가 만료되면 재선거가 실시됩니다. 국회의원은 국민을 대표하여 법률을 제정하고 국가의 중요한 사안에 관여하는 역할을 수행합니다.


로컬 임베딩 모델

In [50]:
!pip install -U langchain-community

In [51]:
import langchain
from langchain.embeddings import HuggingFaceEmbeddings

In [52]:
raw_documents = SimpleTextLoader('/content/korea_constitution.txt').load()
text_splitter = SimpleCharacterTextSplitter(chunk_size=10, chunk_overlap=0)
documents = text_splitter.split_document(raw_documents)

spliting...: 100%|██████████| 152/152 [00:00<00:00, 669678.79it/s]


In [53]:
embed_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
print(embed_model)
db = SimpleVectorStore(documents, embed_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
) model_name='jhgan/ko-sbert-sts' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


embedding...: 100%|██████████| 153/153 [00:02<00:00, 58.06it/s]


In [54]:
query = '대통령의 임기는?'
docs = db.similarity_search(query)
docs

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.5381747143541381),
 ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.',
  0.46576743779733915),
 ('제42조 국회의원의 임기는 4년으로 한다.', 0.43761089284397003),
 ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.4304335262639003)]

In [57]:
retriever = db.as_retriever(k=3)
chain = SimpleRetrievalQA(retriever)
answer = chain.invoke(query)

print(">>",answer)

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.5381747143541381), ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.', 0.46576743779733915), ('제42조 국회의원의 임기는 4년으로 한다.', 0.43761089284397003)]
[#0] 0.5381747143541381
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
[#1] 0.46576743779733915
제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.
② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.
③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.
④ 법관의 정년은 법률로 정한다.
[#2] 0.43761089284397003
제42조 국회의원의 임기는 4년으로 한다.
>> 대통령의 임기는 5년입니다. 한 명의 대통령은 최대 1번의 연임이 가능합니다.


In [58]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

while True:
    user_message = input("user >>")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f"AI >> {ai_message}")


user >>국회의원이 하는 일은 무엇이야?
[('제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.', 0.43880492744100785), ('제3장 국회', 0.4367159435104633), ('제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회외에서 책임을 지지 아니한다.', 0.42270953878635986)]
[#0] 0.43880492744100785
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
[#1] 0.4367159435104633
제3장 국회
[#2] 0.42270953878635986
제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회외에서 책임을 지지 아니한다.
AI >> 국회의원은 국민을 대표하여 국가의 법률을 제정하고 개정하며, 예산안을 심의하고 승인하는 역할을 합니다. 또한 국가의 중요한 정책에 대해 토의하고 결정하는 역할을 하며, 정부의 권한을 감독하고 국민의 요구를 정부에 전달하는 중요한 역할을 합니다. 국회의원은 국민의 명백한 의지와 요구를 받아들여 국가 발전과 국민행복을 위해 노력해야 합니다.
user >>quit
